# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
!pip install pandas
!pip install requests
!pip install bs4

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
ibm-cloud-sdk-core 3.9.0 has requirement PyJWT<3.0.0,>=2.0.1, but you'll have pyjwt 1.7.1 which is incompatible.
ibm-cloud-sdk-core 3.9.0 has requirement requests<3.0,>=2.20, but you'll have requests 2.19.1 which is incompatible.
You are using pip version 10.0.1, however version 21.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
ibm-cloud-sdk-core 3.9.0 has requirement PyJWT<3.0.0,>=2.0.1, but you'll have pyjwt 1.7.1 which is incompatible.
ibm-cloud-sdk-core 3.9.0 has requirement requests<3.0,>=2.20, but you'll have requests 2.19.1 which is incompatible.
You are using pip version 10.0.1, however version 21.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
ibm-cloud-sdk-core 3.9.0 has requirement PyJWT<3.0.0,>=2.0.1, but you'll have pyjwt 1.7.1 which is incompatible.
ibm-cloud-sdk-core 3.9.0 has requirement requests<3.0,>=2.20, but you'll have requests 2.19.1 which is incompatible.
You are using pip version 10.0.1, however version 21.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import numpy as np # library to handle data in a vectorized manner
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

html_data = requests.get(url).text

In [3]:
soup = BeautifulSoup(html_data,"html5lib")

In [4]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                     'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                     'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                     'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [5]:
df.head()

,Borough,Neighborhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Regent Park, Harbourfront",M5A
3,North York,"Lawrence Manor, Lawrence Heights",M6A
4,Queen's Park,Ontario Provincial Government,M7A


In [6]:
df.shape

(103, 3)

In [7]:
df_postcode = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()

df_postcode.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
geolocator = Nominatim(user_agent="toronto_explorer")

lat_col = []
lon_col = []

for postal_code in df['Borough']:
        
    location = geolocator.geocode(postal_code + ', Toronto')
    latitude = location.latitude
    longitude = location.longitude   
    
    lat_col.append(latitude)
    lon_col.append(longitude)

# Insert the Latitude and Longitude columns to the dataframe df
df['Latitude']  = lat_col
df['Longitude'] = lon_col

In [10]:
df.head()

,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,North York,Parkwoods,M3A,43.754326,-79.449117
1,North York,Victoria Village,M4A,43.754326,-79.449117
2,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654174,-79.380812
3,North York,"Lawrence Manor, Lawrence Heights",M6A,43.754326,-79.449117
4,Queen's Park,Ontario Provincial Government,M7A,43.659659,-79.390340


In [15]:
df.dtypes

Borough          object
Neighborhood     object
PostalCode       object
Latitude        float64
Longitude       float64
dtype: object

In [22]:
toronto_data = df[df['Borough'].str.contains('Toronto', regex=False, case=False, na=False)].reset_index(drop=True)
toronto_data.head()

,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654174,-79.380812
1,Downtown Toronto,"Garden District, Ryerson",M5B,43.654174,-79.380812
2,Downtown Toronto,St. James Town,M5C,43.654174,-79.380812
3,East Toronto,The Beaches,M4E,43.626243,-79.396962
4,Downtown Toronto,Berczy Park,M5E,43.654174,-79.380812


In [34]:
toronto_data.shape

(39, 5)

In [23]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [42]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [46]:
CLIENT_ID = 'T2Z2EXSV5R0BXRK3CB2S0U5D0SX1JRXBJRJCVMFDBVSITWSO' # your Foursquare ID
CLIENT_SECRET = 'CKZJEJIU0OZK2ZJ5VQRHHUO4S3OKXBR2NY4Q4Y4SKLZH0X4W' # your Foursquare Secret
ACCESS_TOKEN = 'F3BQMM1O2VAJ201SFO1VP5S2G3UFJPWU15VZJHHMP1EELFET' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T2Z2EXSV5R0BXRK3CB2S0U5D0SX1JRXBJRJCVMFDBVSITWSO
CLIENT_SECRET:CKZJEJIU0OZK2ZJ5VQRHHUO4S3OKXBR2NY4Q4Y4SKLZH0X4W


In [50]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [51]:
# type your answer here
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [52]:
print(toronto_venues.shape)
toronto_venues.head()

(3125, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654174,-79.380812,Elgin And Winter Garden Theatres,43.653394,-79.378507,Theater
1,"Regent Park, Harbourfront",43.654174,-79.380812,UNIQLO ユニクロ,43.655910,-79.380641,Clothing Store
2,"Regent Park, Harbourfront",43.654174,-79.380812,Indigo,43.653515,-79.380696,Bookstore
3,"Regent Park, Harbourfront",43.654174,-79.380812,Ed Mirvish Theatre,43.655102,-79.379768,Theater
4,"Regent Park, Harbourfront",43.654174,-79.380812,CF Toronto Eaton Centre,43.654447,-79.380952,Shopping Mall


In [53]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,93,93,93,93,93,93
"Brockton, Parkdale Village, Exhibition Place",96,96,96,96,96,96
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",93,93,93,93,93,93
Central Bay Street,93,93,93,93,93,93
Christie,93,93,93,93,93,93
Church and Wellesley,93,93,93,93,93,93
"Commerce Court, Victoria Hotel",93,93,93,93,93,93
Davisville,96,96,96,96,96,96
Davisville North,96,96,96,96,96,96


In [55]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 88 uniques categories.


In [56]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Vietnamese Restaurant,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Bakery,Bank,Bar,Bookstore,Breakfast Spot,...,Smoothie Shop,Speakeasy,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Tanning Salon,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Game Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
toronto_onehot.shape

(3125, 88)

In [60]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Vietnamese Restaurant,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Bakery,Bank,Bar,Bookstore,...,Smoothie Shop,Speakeasy,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Tanning Salon,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Game Store
0,Berczy Park,0.010753,0.010753,0.010753,0.010753,0.000000,0.000000,0.010753,0.021505,0.010753,...,0.010753,0.000000,0.010753,0.000000,0.010753,0.021505,0.000000,0.021505,0.000000,0.010753
1,"Brockton, Parkdale Village, Exhibition Place",0.010417,0.020833,0.000000,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,...,0.010417,0.010417,0.000000,0.010417,0.020833,0.010417,0.020833,0.020833,0.010417,0.010417
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.010753,0.010753,0.010753,0.010753,0.000000,0.000000,0.010753,0.021505,0.010753,...,0.010753,0.000000,0.010753,0.000000,0.010753,0.021505,0.000000,0.021505,0.000000,0.010753
3,Central Bay Street,0.010753,0.010753,0.010753,0.010753,0.000000,0.000000,0.010753,0.021505,0.010753,...,0.010753,0.000000,0.010753,0.000000,0.010753,0.021505,0.000000,0.021505,0.000000,0.010753
4,Christie,0.010753,0.010753,0.010753,0.010753,0.000000,0.000000,0.010753,0.021505,0.010753,...,0.010753,0.000000,0.010753,0.000000,0.010753,0.021505,0.000000,0.021505,0.000000,0.010753


In [61]:
toronto_grouped.shape

(39, 88)

In [62]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.11
1  Clothing Store  0.08
2           Hotel  0.03
3           Diner  0.03
4      Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0     Coffee Shop  0.08
1  Clothing Store  0.07
2           Hotel  0.04
3      Restaurant  0.03
4            Café  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
            venue  freq
0     Coffee Shop  0.11
1  Clothing Store  0.08
2           Hotel  0.03
3           Diner  0.03
4      Restaurant  0.03


----Central Bay Street----
            venue  freq
0     Coffee Shop  0.11
1  Clothing Store  0.08
2           Hotel  0.03
3           Diner  0.03
4      Restaurant  0.03


----Christie----
            venue  freq
0     Coffee Shop  0.11
1  Clothing Store  0.08
2           Hotel  0.03
3           Diner  0.03
4      Restaurant  0.03


----Church and Wellesley----
         

In [63]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [66]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Clothing Store,Restaurant,Hotel,Diner,Sandwich Place,Café,Electronics Store,Plaza,Cosmetics Shop
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Clothing Store,Hotel,Café,Restaurant,Salad Place,Electronics Store,Cosmetics Shop,Plaza,Pizza Place
2,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Clothing Store,Restaurant,Hotel,Diner,Sandwich Place,Café,Electronics Store,Plaza,Cosmetics Shop
3,Central Bay Street,Coffee Shop,Clothing Store,Restaurant,Hotel,Diner,Sandwich Place,Café,Electronics Store,Plaza,Cosmetics Shop
4,Christie,Coffee Shop,Clothing Store,Restaurant,Hotel,Diner,Sandwich Place,Café,Electronics Store,Plaza,Cosmetics Shop


In [67]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 3, 4, 4, 4, 4, 4, 3, 3, 3])

In [68]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,PostalCode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654174,-79.380812,4,Coffee Shop,Clothing Store,Restaurant,Hotel,Diner,Sandwich Place,Café,Electronics Store,Plaza,Cosmetics Shop
1,Downtown Toronto,"Garden District, Ryerson",M5B,43.654174,-79.380812,4,Coffee Shop,Clothing Store,Restaurant,Hotel,Diner,Sandwich Place,Café,Electronics Store,Plaza,Cosmetics Shop
2,Downtown Toronto,St. James Town,M5C,43.654174,-79.380812,4,Coffee Shop,Clothing Store,Restaurant,Hotel,Diner,Sandwich Place,Café,Electronics Store,Plaza,Cosmetics Shop
3,East Toronto,The Beaches,M4E,43.626243,-79.396962,1,Performing Arts Venue,Intersection,Comic Shop,Concert Hall,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner
4,Downtown Toronto,Berczy Park,M5E,43.654174,-79.380812,4,Coffee Shop,Clothing Store,Restaurant,Hotel,Diner,Sandwich Place,Café,Electronics Store,Plaza,Cosmetics Shop


In [69]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,The Beaches,1,Performing Arts Venue,Intersection,Comic Shop,Concert Hall,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner
12,"The Danforth West, Riverdale",1,Performing Arts Venue,Intersection,Comic Shop,Concert Hall,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner
15,"India Bazaar, The Beaches West",1,Performing Arts Venue,Intersection,Comic Shop,Concert Hall,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner
17,Studio District,1,Performing Arts Venue,Intersection,Comic Shop,Concert Hall,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner
38,Enclave of M4L,1,Performing Arts Venue,Intersection,Comic Shop,Concert Hall,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner


In [72]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,The Danforth East,2,Park,Pastry Shop,Intersection,Pub,Liquor Store,Diner,Comic Shop,Concert Hall,Cosmetics Shop,Cupcake Shop


In [73]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Dufferin, Dovercourt Village",3,Coffee Shop,Clothing Store,Hotel,Café,Restaurant,Salad Place,Electronics Store,Cosmetics Shop,Plaza,Pizza Place
11,"Little Portugal, Trinity",3,Coffee Shop,Clothing Store,Hotel,Café,Restaurant,Salad Place,Electronics Store,Cosmetics Shop,Plaza,Pizza Place
14,"Brockton, Parkdale Village, Exhibition Place",3,Coffee Shop,Clothing Store,Hotel,Café,Restaurant,Salad Place,Electronics Store,Cosmetics Shop,Plaza,Pizza Place
18,Lawrence Park,3,Coffee Shop,Clothing Store,Hotel,Café,Restaurant,Salad Place,Electronics Store,Cosmetics Shop,Plaza,Pizza Place
19,Roselawn,3,Coffee Shop,Clothing Store,Hotel,Café,Restaurant,Salad Place,Electronics Store,Cosmetics Shop,Plaza,Pizza Place
20,Davisville North,3,Coffee Shop,Clothing Store,Hotel,Café,Restaurant,Salad Place,Electronics Store,Cosmetics Shop,Plaza,Pizza Place
21,Forest Hill North & West,3,Coffee Shop,Clothing Store,Hotel,Café,Restaurant,Salad Place,Electronics Store,Cosmetics Shop,Plaza,Pizza Place
22,"High Park, The Junction South",3,Coffee Shop,Clothing Store,Hotel,Café,Restaurant,Salad Place,Electronics Store,Cosmetics Shop,Plaza,Pizza Place
23,North Toronto West,3,Coffee Shop,Clothing Store,Hotel,Café,Restaurant,Salad Place,Electronics Store,Cosmetics Shop,Plaza,Pizza Place
24,"The Annex, North Midtown, Yorkville",3,Coffee Shop,Clothing Store,Hotel,Café,Restaurant,Salad Place,Electronics Store,Cosmetics Shop,Plaza,Pizza Place


In [74]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",4,Coffee Shop,Clothing Store,Restaurant,Hotel,Diner,Sandwich Place,Café,Electronics Store,Plaza,Cosmetics Shop
1,"Garden District, Ryerson",4,Coffee Shop,Clothing Store,Restaurant,Hotel,Diner,Sandwich Place,Café,Electronics Store,Plaza,Cosmetics Shop
2,St. James Town,4,Coffee Shop,Clothing Store,Restaurant,Hotel,Diner,Sandwich Place,Café,Electronics Store,Plaza,Cosmetics Shop
4,Berczy Park,4,Coffee Shop,Clothing Store,Restaurant,Hotel,Diner,Sandwich Place,Café,Electronics Store,Plaza,Cosmetics Shop
5,Central Bay Street,4,Coffee Shop,Clothing Store,Restaurant,Hotel,Diner,Sandwich Place,Café,Electronics Store,Plaza,Cosmetics Shop
6,Christie,4,Coffee Shop,Clothing Store,Restaurant,Hotel,Diner,Sandwich Place,Café,Electronics Store,Plaza,Cosmetics Shop
7,"Richmond, Adelaide, King",4,Coffee Shop,Clothing Store,Restaurant,Hotel,Diner,Sandwich Place,Café,Electronics Store,Plaza,Cosmetics Shop
10,"Harbourfront East, Union Station, Toronto Islands",4,Coffee Shop,Clothing Store,Restaurant,Hotel,Diner,Sandwich Place,Café,Electronics Store,Plaza,Cosmetics Shop
13,"Toronto Dominion Centre, Design Exchange",4,Coffee Shop,Clothing Store,Restaurant,Hotel,Diner,Sandwich Place,Café,Electronics Store,Plaza,Cosmetics Shop
16,"Commerce Court, Victoria Hotel",4,Coffee Shop,Clothing Store,Restaurant,Hotel,Diner,Sandwich Place,Café,Electronics Store,Plaza,Cosmetics Shop
